# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Set Up Environment

In [2]:
network = "local"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'custom_contracts_20201110125748.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [3]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [4]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [5]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [12]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [11]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

1000000000000000000000000

### Importing wallet from keystore file

In [8]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

787640640000000000000000000

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [13]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
#Run this when you need to zip the contract codes (before deploying them)
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            # make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name 


contracts

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'},
 'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider'}}

In [9]:


pprint(contracts)


{'addressProvider': {'SCORE': 'cx42dbafcf33b26b0cc7f0e1f7ca377bfeebc6c42e',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'},
 'feeProvider': {'SCORE': 'cxd66f32f94600843c9441f504ba6ee075d4387bc8',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPool': {'SCORE': 'cx83e2b9cfa3bc5e0836e8ea48dc4b67f47723a1b1',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'lendingPoolCore': {'SCORE': 'cx5950ae341b07b9d557ed545e45e4f2e3a506ad9a',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'lendingPoolDataProvider': {'SCORE': 'cxf6102cd8af9c3facd6607c286bd14630de92d976',
                             'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'oToken': {'SCORE': 'cx5e71af8b721dfe96af95f03cb514bfa04480180a',
            'zip': '/home/shubham/Work

# Deploying all contracts


In [14]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
lendingPoolDataProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
lendingPool {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
feeProvider {'

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider',
  'SCORE': 'cx5e3ebd83a9528fb6222019e7e7b846a9fb87d3fb'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool',
  'SCORE': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider',
  'SCORE': 'cxfb57ede59109660ee7a48c633f0d4619368a5469'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore',
  'SCORE': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken',
  'SCORE': 'cx705da06a4ca03eb4f4b383db07e997fb7129b131'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle',
  'SCORE': 'cx8b821d255f6c594a872cc210a1698901a31f92fc'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token'

# Updating a contract

In [42]:
contract = 'lendingPoolDataProvider'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x2a79f8c00b7d3e69a26f2f92fa55ea38113265a135585b3d5b5de3ee13b162d9',
 'blockHeight': 4736,
 'blockHash': '0xa02432576f9310055698fbae6d22cdf874892c5ed915405085ca7fcd9781f57b',
 'txIndex': 1,
 'to': 'cx5e3ebd83a9528fb6222019e7e7b846a9fb87d3fb',
 'scoreAddress': 'cx5e3ebd83a9528fb6222019e7e7b846a9fb87d3fb',
 'stepUsed': 1712194400,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1712194400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Configuring SCOREs's parameters

In [15]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 25*10**14}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"1{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"5{'0'*17}"} ]}}]

In [16]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

ockHeight': 1428, 'blockHash': '0x3093f0ccf0067e4a06547c1ca4fb0ff044e72a19fc57ba430cc6e65e3c66054b', 'txIndex': 1, 'to': 'cx5e3ebd83a9528fb6222019e7e7b846a9fb87d3fb', 'stepUsed': 150920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 150920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Adding new reserve in LendingPoolCore



In [17]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"600000000000000000","liquidationThreshold":"6500000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xdf16c5c0365f8b62cc2aeba10ec27551b6fa123e2037c659d98525134fcb254c',
 'blockHeight': 1444,
 'blockHash': '0x98315aef624555788d3b4a1904fdf5361893ef3dc42d00fa9d4de80d0d75fa2c',
 'txIndex': 1,
 'to': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
 'stepUsed': 273040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 273040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Transfering USDb to a wallet

In [ ]:
params = {"_to": "", "_value": 1000000000000000000000,}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing USDb from a wallet

In [18]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xa1e71e86c7dddeb51aa314f1e7d5414d0e677ed93f2da5bd7d1249f2a954c756',
 'blockHeight': 1460,
 'blockHash': '0xb62720a9917946e9bfb9e7e2b5b1fe59247e1b865e9ca674d0c2102141c26f6d',
 'txIndex': 1,
 'to': 'cx1592d568457a2881e4bd36117a8295ac2a000377',
 'stepUsed': 769320,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 769320,
 'eventLogs': [{'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 593',
    '0x3635c9adc5dea00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'Utilization rate core line 589',
    '0x0',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c28

# Borrowing USDb

In [43]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d', 'stepLimit': '0x14bbf8', 'timestamp': '0x5b46dd9bf37cd', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx1592d568457a2881e4bd36117a8295ac2a000377', '_amount': '0x8ac7230489e80000'}}, 'signature': 'Folqgh9js3lLO0q/dtvGledD0lxOz/LirI6qn2i7J4AWTIR7p2T4cxqoKL4lfDXYE9q5cEPlAjdSF6pWXzwJlgA='}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x8dd5715179fb035028e00749c33effa1d3efd06c4955ec71504df6cc145dd53b',
 'blockHeight': 4799,
 'blockHash': '0xbce5c543a5e8b5bd12ed8adf8606a2b135bbe0638fd69bfa816dcddc3651ead2',
 'txIndex': 1,
 'to': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
 'stepUsed': 1258840,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1258840,
 'eventLogs': [{'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'available liquidity at pool line 245',
    '0x2e141ea081ca080000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'data at pool line 256',
    '0x3635dd4440d4c9f9a8',
    '0x8274de6e63dbd6b43'],
   'data': ['0x0']},
  {'scoreAddress': 'cx288685685cf0a8b8f7653c00f6dc42c8406f0d5d',
   'indexed': ['PrintData(str,int,int,int)',
    'amout of collateral needed USD pool line 268',
    '0x2519f35d8353d2db',
    '0x0'],
   'data': ['0x0']},

# Reedeming/Withdrawing USDb

In [ ]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

In [46]:
params ={"_reserveAddress": contracts['sample_token']['SCORE'],"_baseLTVasCollateral" :6*10**17}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26', 'stepLimit': '0x3ea80', 'timestamp': '0x5b46e394124c5', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cx1592d568457a2881e4bd36117a8295ac2a000377', '_baseLTVasCollateral': '0x853a0d2313c0000'}}, 'signature': '6sQbRORx8Fy9fwWP3Y7mMIt+tjQnXNDT5LRbcWpqGQECq17MGzd3eC+Ut2zav9sGyXIyZsDnbPSqBQYPsAEMhwE='}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x71cf3150013665071a110083bf2206a8f3c7a57dac3a38b9d0b00900c5fe3f1c',
 'blockHeight': 5599,
 'blockHash': '0xe2ee9d6d1bdfe3c55a02925c44dd522d64746efb5cb2540d0497b272bb7287a8',
 'txIndex': 1,
 'to': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
 'stepUsed': 156640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 156640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Repaying USDb

In [38]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 10000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 10000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa00f366c3e3bc59fa930e6c4134a4de0ab0437250175baf3a3342ac194d7049a',
 'blockHeight': 2668,
 'blockHash': '0x51bc669d21d2e946367aea11c77091441d579d42044fe1abaf3842a27ded2a41',
 'txIndex': 1,
 'to': 'cx1592d568457a2881e4bd36117a8295ac2a000377',
 'stepUsed': 951460,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 951460,
 'eventLogs': [{'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 408',
    '0x853a0d2313c00000',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'params check core line 593',
    '0x2e0e918aa053e00000',
    '0x82738b02dc459515c'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd2efb802be81bdca98f3024a35c283046a0d9c26',
   'indexed': ['PrintData(str,int,int,int)',
    'Utilization rate core line 598',
    '0x216541f7accc0ec',
    '0x0'],
   'data': ['0x0']},
  {'scoreAddress': 'cxd

# Readonly methods to get data

### Getting specific reserve data

In [47]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 840.0,
 'baseLTVasCollateral': 0.6,
 'borrowCumulativeIndex': 1.0000407265079532,
 'borrowRate': 0.020689400501447817,
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1605758782881092,
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 65.0,
 'liquidityCumulativeIndex': 1.0000055134794177,
 'liquidityRate': 0.002985623389280219,
 'oTokenAddress': 'cx705da06a4ca03eb4f4b383db07e997fb7129b131',
 'reserveAddress': 'cx1592d568457a2881e4bd36117a8295ac2a000377',
 'totalBorrows': 160.40612620691502,
 'totalLiquidity': 1000.406126206915,
 'usageAsCollateralEnabled': '0x1'}


### Getting data of all reserves

In [50]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': 60.0,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': '0x0',
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x0',
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 65.0,
          'liquidityCumulativeIndex': 1.0,
          'liquidityRate': '0x0',
          'oTokenAddress': 'cx5e71af8b721dfe96af95f03cb514bfa04480180a',
          'reserveAddress': 'cx3975f669579804bb7e05e954a66626095a5079f8',
          'totalBorrows': '0x0',
          'totalLiquidity': '0x0',
          'usageAsCollateralEnabled': '0x1'}}


### Getting user account data

In [48]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 65.0,
 'currentLtv': 0.6,
 'healthFactor': 405.1601486659374,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 160.40629763562876,
 'totalCollateralBalanceUSD': 1000.0056677033939,
 'totalFeesUSD': 0.025,
 'totalLiquidityBalanceUSD': 1000.0056677033939}


In [53]:
#returns the user data of all the reserves combined
params = {'_reserve': contracts['sample_token']['SCORE'],'_amount':10*10**18,"_fee":1,'_userCurrentBorrowBalanceUSD':0,'_userCurrentFeesUSD':0,'_userCurrentLtv':5*10**17}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("calculateCollateralNeededUSD")\
    .params(params)\
    .build()
response = icon_service.call(_call)
response = int(response,0)/10**18
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
print(response)

20.0


### Getting user reserve data

In [41]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.020026675449629917,
 'currentBorrowBalance': 150.40016098810878,
 'currentBorrowBalanceUSD': 150.40016098810878,
 'currentOTokenBalance': 1000.000144889243,
 'currentOTokenBalanceUSD': 1000.000144889243,
 'lastUpdateTimestamp': 1605696265046831,
 'liquidityRate': 0.002710813163424891,
 'originationFee': 0.0,
 'principalBorrowBalance': 150.4001550664656,
 'principalBorrowBalanceUSD': 150.4001550664656,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 1.0000010247764899}


### Getting the data of user for all reserves

In [51]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0}}


### Get reserve configurations


In [53]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 60000000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 65000000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [56]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 60000000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 65000000000000000000,
          'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511